In [1]:
from src.inspector_git.iglog.readers.ig_log_reader import IGLogReader

def main():
    path_inspector_git = "../../test-input/inspector-git/zeppelin.iglog"

    reader = IGLogReader()

    with open(path_inspector_git, "r", encoding="utf-8") as stream:
        git_log = reader.read(stream)

    print(f"Număr commit-uri citite: {len(git_log.commits)}")
    for idx, commit in enumerate(git_log.commits[:5], start=1):  # afișăm doar primele 5 pentru test
        print(f"\nCommit {idx}:")
        print(f"  ID: {commit.id}")
        print(f"  Author: {commit.author_name} <{commit.author_email}>")
        print(f"  Date: {commit.author_date}")
        print(f"  Message: {commit.message.strip() if commit.message else ''}")

    return git_log.commits

commits = main()


Număr commit-uri citite: 5512

Commit 1:
  ID: ec716cf6e8725dc456db42d11a1972aa78d9de91
  Author: Lee moon soo <leemoonsoo@gmail.com>
  Date: Tue Jun 18 21:40:59 2013 -0700
  Message: Initial commit

Commit 2:
  ID: d512bdeb12010d94f47b464d2e424d3ae03fa7d0
  Author: anthony corbacho <corbacho.anthony@gmail.com>
  Date: Wed Jun 19 15:35:47 2013 +0900
  Message: Update .gitignore

Commit 3:
  ID: d55722b80cec76fc6df316ebbfac7751b2747c7d
  Author: Lee moon soo <leemoonsoo@gmail.com>
  Date: Fri Jul 5 15:42:12 2013 +0900
  Message: Add mailing list

Commit 4:
  ID: e02e99d88265d12abf1d7109d5d799bdcba7f64f
  Author: Lee moon soo <leemoonsoo@gmail.com>
  Date: Sun Jul 7 10:31:05 2013 +0900
  Message: Adding License

Commit 5:
  ID: c20fd6454e5733628ead3f129c7e803c1ead5b94
  Author: Lee moon soo <leemoonsoo@gmail.com>
  Date: Thu Aug 15 20:57:35 2013 +0900
  Message: Some code is ported from NFlabs Peloton (www.nflabs.com)


In [3]:
from pprint import pprint
pprint(commits[0].changes[0])

ChangeDTO(old_file_name='dev/null',
          new_file_name='.gitignore',
          type=<ChangeType.ADD: 'ADD'>,
          parent_commit_id='',
          is_binary=False)


# Imports

In [1]:
from src.inspector_git import IGLogReader, GitLogDTO
from src.jira_miner.models import JsonFileFormatJira
from src.github_miner import JsonFileFormatGithub
import json
from pathlib import Path
from pprint import pprint
import re
from datetime import datetime
import pickle
from graph import *

In [4]:
import os

# Print the current working directory
print(os.getcwd())


/home/vortex/Work/BachelorThesis/Vortex/src/notebooks


# Load data from SERIALIZED formats

In [6]:
# JSON
path_jira = "../../test-input/jira-miner/ZEPPELIN-detailed-issues.json"
path_github = "../../test-input/github-miner/githubProject.json"
# IGLOG
path_inspector_git = "../../test-input/inspector-git/zeppelin.iglog"



def load_from_json(model_cls, file_path: str):
    """Load JSON from a given path and validate it with the provided model class."""
    file_path = Path(file_path)

    if not file_path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    with file_path.open("r", encoding="utf-8") as f:
        data = json.load(f)

    return model_cls.model_validate(data)

def load_jsons():
    jira_data_loaded = load_from_json(JsonFileFormatJira, path_jira)
    github_data_loaded = load_from_json(JsonFileFormatGithub, path_github)
    return jira_data_loaded, github_data_loaded

ig_log_reader = IGLogReader()

jira_data, github_data = load_jsons()
inspector_git_data = ig_log_reader.read(path_inspector_git)

In [3]:
import importlib, graph
importlib.reload(graph)
from graph import *

graph = Graph()

graph.add_inspector_git_data(inspector_git_data)
graph.add_jira_data(jira_data)
graph.add_github_data(github_data)

graph.link_issues_with_git_commits()
graph.link_pull_request_with_issue(jira_data)
graph.link_pull_requests_with_git_commits()

print(graph.summary())

Indexing is 6186
~~~~ Graph summary ~~~~
commits: 5512
git_users: 602
files: 6185

issue_statuses: 48
issue_types: 7
issue_status_categories: 3
jira_users: 2008
issues: 6202

pull_requests: 5022
git_hub_users: 690
git_hub_commits: 17869

nodes: 44148
edges: 246520



In [5]:
graph.files = []
graph.issues = []
graph.save()

RecursionError: maximum recursion depth exceeded while pickling an object

In [5]:
fisiere = [f.path for f in graph.files]
strings_sorted = sorted(fisiere)
# Save to file
with open("recursive.txt", "w") as f:
    for item in strings_sorted:
        f.write(item + "\n")

In [38]:
nr  =0
for file in graph.files:
    if file.path == "/dev/null":
        nr += 1

print(nr)

2270
